In [ ]:
import re
import numpy as np

with open('input.txt', 'r') as f:
  lines = f.read().splitlines()

In [ ]:
M = np.array([list(l) for l in lines])

# 2D 90 degree clockwise rotation.
rot_right = np.array([
   [ 0, 1],
   [-1, 0]])

# Helper function to check if a position is still on the map.
def in_bounds(M, pos):
  return 0 <= pos[0] < M.shape[0] and 0 <= pos[1] < M.shape[1]

# Helper function to check if a given position on the map is an obstacle.
def is_obstacle(M, pos):
  return M[pos[0], pos[1]] == '#'

# Initial position and direction vector.
start_pos = np.array(np.where(M == '^')).flatten()
start_v = np.array([-1, 0])

def trace(M, pos, v):
  # Counts the number of times each position was visited.
  C = np.zeros(M.shape)

  while in_bounds(M, pos):
    # Mark cell as visited.
    C[pos[0], pos[1]] += 1

    # Bail if we're visiting this spot for the fourth time, which means that we're trapped in a cycle
    # for sure (relevant for puzzle part 2).
    if C[pos[0], pos[1]] >= 4:
      return C

    # Check what's ahead of us. If it's an obstacle, rotate right, otherwise take the step forward.
    pos_prime = pos + v
    if in_bounds(M, pos_prime) and is_obstacle(M, pos_prime):
      v = np.matmul(rot_right, v)
    else:
      pos = pos_prime

  return C

# Part 1: count the number of cells the guard ever visits.
C = trace(M, start_pos, start_v)
np.count_nonzero(C)

In [ ]:
n_cycles = 0

# Part 2:
# Iterate over the coverage from part 1, which contains all positions the guard will ever visit.
# For each such position that isn't the starting position itself, place a new obstacle, then re-
# run the trace and check if the result indicates that the guard was trapped in a cycle somewhere.
for i in range(M.shape[0]):
  for j in range(M.shape[1]):
    if C[i, j] > 0 and (i, j) != tuple(start_pos):
      M_prime = M.copy()
      M_prime[i, j] = '#'
      C_prime = trace(M_prime, start_pos, start_v)
      if np.max(C_prime) >= 4:
        n_cycles += 1

n_cycles